In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble,model_selection,feature_extraction,preprocessing,linear_model,svm
import lightgbm
import xgboost
import scipy
from bayes_opt import BayesianOptimization
%matplotlib inline
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
import catboost

In [2]:
train=pd.read_csv("./ml5/train.csv",sep=";")
test=pd.read_csv("./ml5/test.csv",sep=";")
sub=pd.read_csv("./ml5example.csv",header=None)

In [3]:
train.groupby(["cholesterol","active"]).size()

cholesterol  active
1            0         10353
             1         42032
2            0          1919
             1          7630
3            0          1467
             1          6599
dtype: int64

In [4]:
X=train.copy()
X.drop("id",axis=1,inplace=True)
ts=test.copy()
ts.drop("id",axis=1,inplace=True)
y=train.cardio

In [5]:
X["halat"]=0
ts["halat"]=0

In [6]:
X["trash_hi"]=X.ap_hi
X["trash_lo"]=X.ap_lo

ts["trash_hi"]=ts.ap_hi
ts["trash_lo"]=ts.ap_lo

X.loc[(X.trash_hi<X.trash_lo),"halat"]=1
X.loc[(X.trash_hi<X.trash_lo),"ap_lo"]=X[(X.trash_hi<X.trash_lo)]['trash_hi']#менял местами верхнее и нежное
X.loc[(X.trash_hi<X.trash_lo),"ap_hi"]=X[(X.trash_hi<X.trash_lo)]['trash_lo']

ts.loc[(ts.trash_hi<ts.trash_lo),"halat"]=1
ts.loc[(ts.trash_hi<ts.trash_lo),"ap_lo"]=ts[(ts.trash_hi<ts.trash_lo)]['trash_hi']#менял местами верхнее и нежное
ts.loc[(ts.trash_hi<ts.trash_lo),"ap_hi"]=ts[(ts.trash_hi<ts.trash_lo)]['trash_lo']

X.drop('trash_hi',axis=1,inplace=True)
X.drop('trash_lo',axis=1,inplace=True)

ts.drop('trash_hi',axis=1,inplace=True)
ts.drop('trash_lo',axis=1,inplace=True)


In [7]:
X[X["ap_hi"]<0]["halat"]=1
X["ap_hi"]=np.abs(train.ap_hi)#правим верхнее давление

X.loc[X.ap_hi>300,"halat"]=1
X.loc[X.ap_hi>300,"ap_hi"]=X[X["ap_hi"]>300]["ap_hi"]/10
X.loc[X.ap_hi>300,"ap_hi"]=X[X["ap_hi"]>300]["ap_hi"]/10

X.loc[X.ap_hi<30,"halat"]=1
X.loc[X.ap_hi<30,"ap_hi"]=X[X["ap_hi"]<30]['ap_hi']*10
X.loc[X.ap_hi<30,"ap_hi"]=X[X["ap_hi"]<30]['ap_hi']*10


ts["ap_hi"]=np.abs(ts.ap_hi)#правим верхнее давление

ts.loc[ts.ap_hi>300,"halat"]=1
ts.loc[ts.ap_hi>300,"ap_hi"]=ts[ts["ap_hi"]>300]["ap_hi"]/10

ts.loc[ts.ap_hi>300,"halat"]=1
ts.loc[ts.ap_hi>300,"ap_hi"]=ts[ts["ap_hi"]>300]["ap_hi"]/10

ts.loc[ts.ap_hi<30,"halat"]=1
ts.loc[ts.ap_hi<30,"ap_hi"]=ts[ts["ap_hi"]<30]['ap_hi']*10

ts.loc[ts.ap_hi<30,"halat"]=1
ts.loc[ts.ap_hi<30,"ap_hi"]=ts[ts["ap_hi"]<30]['ap_hi']*10

In [8]:
X[X["ap_lo"]<0]["halat"]=1
X["ap_lo"]=np.abs(train.ap_lo)#и нежное

X.loc[X.ap_lo>200,"halat"]=1
X.loc[X.ap_lo>200,"ap_lo"]=X[X["ap_lo"]>200]['ap_lo']/10
X.loc[X.ap_lo>200,"ap_lo"]=X[X["ap_lo"]>200]['ap_lo']/10

X.loc[X.ap_lo<30,"halat"]=1
X.loc[X.ap_lo<30,"ap_lo"]=X[X["ap_lo"]<30]['ap_lo']*10
X.loc[X.ap_lo<30,"ap_lo"]=X[X["ap_lo"]<30]['ap_lo']*10


ts[ts["ap_lo"]<0]["halat"]=1
ts["ap_lo"]=np.abs(ts.ap_lo)#и нежное

ts.loc[ts.ap_lo>200,"halat"]=1
ts.loc[ts.ap_lo>200,"ap_lo"]=ts[ts["ap_lo"]>200]['ap_lo']/10
ts.loc[ts.ap_lo>200,"ap_lo"]=ts[ts["ap_lo"]>200]['ap_lo']/10

ts.loc[ts.ap_lo<30,"halat"]=1
ts.loc[ts.ap_lo<30,"ap_lo"]=ts[ts["ap_lo"]<30]['ap_lo']*10
ts.loc[ts.ap_lo<30,"ap_lo"]=ts[ts["ap_lo"]<30]['ap_lo']*10

In [9]:
X["trash_hi"]=X.ap_hi
X["trash_lo"]=X.ap_lo

ts["trash_hi"]=ts.ap_hi
ts["trash_lo"]=ts.ap_lo

X.loc[(X.trash_hi<X.trash_lo),"halat"]=1
X.loc[(X.trash_hi<X.trash_lo),"ap_lo"]=X[(X.trash_hi<X.trash_lo)]['trash_hi']#менял местами верхнее и нежное
X.loc[(X.trash_hi<X.trash_lo),"ap_hi"]=X[(X.trash_hi<X.trash_lo)]['trash_lo']

ts.loc[(ts.trash_hi<ts.trash_lo),"halat"]=1
ts.loc[(ts.trash_hi<ts.trash_lo),"ap_lo"]=ts[(ts.trash_hi<ts.trash_lo)]['trash_hi']#менял местами верхнее и нежное
ts.loc[(ts.trash_hi<ts.trash_lo),"ap_hi"]=ts[(ts.trash_hi<ts.trash_lo)]['trash_lo']

X.drop('trash_hi',axis=1,inplace=True)
X.drop('trash_lo',axis=1,inplace=True)

ts.drop('trash_hi',axis=1,inplace=True)
ts.drop('trash_lo',axis=1,inplace=True)


In [10]:
X["IMT"]=X["weight"]/((X["height"]/100)*(X["height"]/100))

ts["IMT"]=ts["weight"]/((ts["height"]/100)*(ts["height"]/100))

In [11]:
X.loc[(X["weight"]<40),"halat"]=1
X.loc[(X["weight"]<40)&(X["weight"]>=20),"weight"]=X[(X["weight"]<40)][X["weight"]>=20]["weight"]+30#пальцами промазывали на нампаде на кнопку вверх
X.loc[(X["weight"]<20),"weight"]=X[X["weight"]<20]["weight"]*10#ноль забыли

ts.loc[(ts["weight"]<40),"halat"]=1
ts.loc[(ts["weight"]<40)&(ts["weight"]>=20),"weight"]=ts[(ts["weight"]<40)][ts["weight"]>=20]["weight"]+30#пальцами промазывали на нампаде на кнопку вверх
ts.loc[(ts["weight"]<20),"weight"]=ts[ts["weight"]<20]["weight"]*10#ноль забыли

In [12]:
X.loc[(X["height"]>230),"halat"]=1
X.loc[(X["height"]<90),"halat"]=1
X.loc[(X["height"]>230),"height"]=X[X["height"]>230]["height"]-100 #рост хорош
X.loc[(X["height"]<90),"height"]=X[X["height"]<90]["height"]+100

ts.loc[(ts["height"]>230),"halat"]=1
ts.loc[(ts["height"]<90),"halat"]=1
ts.loc[(ts["height"]>230),"height"]=ts[ts["height"]>230]["height"]-100#рост хорош
ts.loc[(ts["height"]<90),"height"]=ts[ts["height"]<90]["height"]+100

In [13]:
X[X["IMT"]>60]["halat"]=1

In [14]:
X["trash_he"]=X.height
X["trash_we"]=X.weight

X.loc[X.IMT>60,"height"]=X[X.IMT>60]['trash_we']#менял местами рост и вес
X.loc[X.IMT>60,"weight"]=X[X.IMT>60]['trash_he']

X.drop('trash_he',axis=1,inplace=True)
X.drop('trash_we',axis=1,inplace=True)
X["IMT"]=X["weight"]/((X["height"]/100)*(X["height"]/100))

X["trash_he"]=X.height
X["trash_we"]=X.weight

X.loc[X.IMT>60,"height"]=X[X.IMT>60]['trash_we']
X.loc[X.IMT>60,"weight"]=X[X.IMT>60]['trash_he']

X.drop('trash_he',axis=1,inplace=True)
X.drop('trash_we',axis=1,inplace=True)
X["IMT"]=X["weight"]/((X["height"]/100)*(X["height"]/100))
########################################################################

ts["trash_he"]=ts.height
ts["trash_we"]=ts.weight

ts.loc[ts.IMT>60,"height"]=ts[ts.IMT>60]['trash_we']#менял местами рост и вес
ts.loc[ts.IMT>60,"weight"]=ts[ts.IMT>60]['trash_he']

ts.drop('trash_he',axis=1,inplace=True)
ts.drop('trash_we',axis=1,inplace=True)
ts["IMT"]=ts["weight"]/((ts["height"]/100)*(ts["height"]/100))

ts["trash_he"]=ts.height
ts["trash_we"]=ts.weight

ts.loc[ts.IMT>60,"height"]=ts[ts.IMT>60]['trash_we']#менял местами рост и вес
ts.loc[ts.IMT>60,"weight"]=ts[ts.IMT>60]['trash_he']

ts.drop('trash_he',axis=1,inplace=True)
ts.drop('trash_we',axis=1,inplace=True)
ts["IMT"]=ts["weight"]/((ts["height"]/100)*(ts["height"]/100))

In [15]:
#X['Soviet_norm_hi']=X["ap_hi"]/(109+X["weight"]*0.1+(X["age"]/365)*0.5)#совет себя не оправдывает
#X['Soviet_norm_low']=X["ap_lo"]/(63+X["weight"]*0.15+(X["age"]/365)*0.1)#всем больше 30 и меньше 65

#ts['Soviet_norm_hi']=ts["ap_hi"]/(109+ts["weight"]*0.1+(ts["age"]/365)*0.5)
#ts['Soviet_norm_low']=ts["ap_lo"]/(63+ts["weight"]*0.15+(ts["age"]/365)*0.1)

In [16]:
X["Ketle"]=(X["weight"]*100)/X['height']
X["Devina"]=np.where(X.gender==1,X.weight/(45.5+2.3*(X.height*0.39-60)),X.weight/(50+2.3*(X.height*0.39-60)))
X["Miller"]=np.where(X.gender==1,X.weight/(53.1+1.36*(X.height*0.39-60)),X.weight/(56.2+1.41*(X.height*0.39-60)))
X["Nagler"]=np.where(X.gender==1,X.weight/(45.3+2.27*(X.height*0.39-60)),X.weight/(48+2.7*(X.height*0.39-60)))
X["Mohhameda"]=X.weight/(X.height**2*0.00225)
X["Hamvi"]=np.where(X.gender==1,X.weight/(45.5+2.2*(X.height*0.39-60)),X.weight/(48+2.7*(X.height*0.39-60)))
X["Kuper"]=np.where(X.gender==1,X.weight/((X.height*3.5/2.54-108)*0.453),X.weight/((X.height*4.0/2.54-128)*0.453))


ts["Ketle"]=(ts["weight"]*100)/ts['height']
ts["Devina"]=np.where(ts.gender==1,ts.weight/(45.5+2.3*(ts.height*0.39-60)),ts.weight/(50+2.3*(ts.height*0.39-60)))
ts["Miller"]=np.where(ts.gender==1,ts.weight/(53.1+1.36*(ts.height*0.39-60)),ts.weight/(56.2+1.41*(ts.height*0.39-60)))
ts["Nagler"]=np.where(ts.gender==1,ts.weight/(45.3+2.27*(ts.height*0.39-60)),ts.weight/(48+2.7*(ts.height*0.39-60)))
ts["Mohhameda"]=ts.weight/(ts.height**2*0.00225)
ts["Hamvi"]=np.where(ts.gender==1,ts.weight/(45.5+2.2*(ts.height*0.39-60)),ts.weight/(48+2.7*(ts.height*0.39-60)))
ts["Kuper"]=np.where(ts.gender==1,ts.weight/((ts.height*3.5/2.54-108)*0.453),ts.weight/((ts.height*4.0/2.54-128)*0.453))

In [17]:
X["Ketle_cat"]=X.Ketle//0.05
X["Devina_cat"]=X.Devina//0.05
X["Nagler_cat"]=X.Nagler//0.05
X["Mohhameda_cat"]=X.Mohhameda//0.05
X["Hamvi_cat"]=X.Hamvi//0.05
X["Kuper_cat"]=X.Kuper//0.05

ts["Ketle_cat"]=X.Ketle//0.05
ts["Devina_cat"]=X.Devina//0.05
ts["Nagler_cat"]=X.Nagler//0.05
ts["Mohhameda_cat"]=X.Mohhameda//0.05
ts["Hamvi_cat"]=X.Hamvi//0.05
ts["Kuper_cat"]=X.Kuper//0.05


In [18]:
len((X["Kuper"]//0.05).unique())

74

In [19]:
X.loc[(X.height==X.ap_lo)|(X.height==X.weight)|(X.ap_hi==X.ap_lo),"halat"]=1

In [20]:
X.loc[X["gluc"]==1,"gluc mean"]=X.loc[X["gluc"]==1,"cardio"].mean()#просто среднее по категориям, классы баланс,можно и без гладких средних
X.loc[X["gluc"]==2,"gluc mean"]=X.loc[X["gluc"]==2,"cardio"].mean()
X.loc[X["gluc"]==3,"gluc mean"]=X.loc[X["gluc"]==3,"cardio"].mean()

X.loc[X["cholesterol"]==1,"cholesterol mean"]=X.loc[X["cholesterol"]==1,"cardio"].mean()
X.loc[X["cholesterol"]==2,"cholesterol mean"]=X.loc[X["cholesterol"]==2,"cardio"].mean()
X.loc[X["cholesterol"]==3,"cholesterol mean"]=X.loc[X["cholesterol"]==3,"cardio"].mean()


ts.loc[ts["gluc"]==1,"gluc mean"]=X.loc[X["gluc"]==1,"cardio"].mean()#просто среднее по категориям, классы баланс,можно и без гладких средних
ts.loc[ts["gluc"]==2,"gluc mean"]=X.loc[X["gluc"]==2,"cardio"].mean()
ts.loc[ts["gluc"]==3,"gluc mean"]=X.loc[X["gluc"]==3,"cardio"].mean()

ts.loc[ts["cholesterol"]==1,"cholesterol mean"]=X.loc[X["cholesterol"]==1,"cardio"].mean()
ts.loc[ts["cholesterol"]==2,"cholesterol mean"]=X.loc[X["cholesterol"]==2,"cardio"].mean()
ts.loc[ts["cholesterol"]==3,"cholesterol mean"]=X.loc[X["cholesterol"]==3,"cardio"].mean()

In [21]:
co=preprocessing.OneHotEncoder()
a=co.fit_transform(train[["cholesterol","gluc"]])
b=co.transform(ts[["cholesterol","gluc"]])

a.toarray().T
b.toarray().T
for i in range(0,6):
    X["ohe{}".format(i+1)]=a.toarray().T[i]
    ts["ohe{}".format(i+1)]=b.toarray().T[i]
    
X.drop("cholesterol",axis=1,inplace=True)
X.drop("gluc",axis=1,inplace=True)

ts.drop("cholesterol",axis=1,inplace=True)
ts.drop("gluc",axis=1,inplace=True)

In [22]:
ts["smoke"]=pd.factorize(ts.smoke)[0]#1 КЛАСС ЭТО НАНЫ, 2 КЛАСС ЭТО ПЕРВЫЙ
ts["alco"]=pd.factorize(ts.alco)[0]#0-0 1-none 2-1
ts["active"]=pd.factorize(ts.active)[0]#0-1 1-0 2-none

In [23]:
ts_med=ts.copy()

ts_med["smoke_tra"]=ts.smoke
ts_med["alco_tra"]=ts.alco
ts_med["active_tra"]=ts.active


ts_med.loc[ts_med.smoke_tra==2,"smoke"]=1
ts_med.loc[ts_med.smoke_tra==0,"smoke"]=0
ts_med.loc[ts_med.smoke_tra==1,"smoke"]=ts_med.loc[ts_med.smoke_tra!=1,"smoke_tra"].median()

ts_med.loc[ts_med.alco_tra==1,"alco"]=ts_med.loc[ts_med.alco_tra!=1,"alco_tra"].median()
ts_med.loc[ts_med.alco_tra==2,"alco"]=1

ts_med.loc[ts_med.active_tra==1,"active"]=0
ts_med.loc[ts_med.active_tra==0,"active"]=1
ts_med.loc[ts_med.active_tra==2,"active"]=ts_med.loc[ts_med.active_tra!=2,"active_tra"].median()


ts_med.drop(["smoke_tra","alco_tra","active_tra"],axis=1,inplace=True)

In [24]:
ts_nan=ts.copy()

ts_nan["smoke_tra"]=ts.smoke
ts_nan["alco_tra"]=ts.alco
ts_nan["active_tra"]=ts.active


ts_nan.loc[ts_nan.smoke_tra==2,"smoke"]=1
ts_nan.loc[ts_nan.smoke_tra==0,"smoke"]=0
ts_nan.loc[ts_nan.smoke_tra==1,"smoke"]=np.NaN

ts_nan.loc[ts_nan.alco_tra==1,"alco"]=np.NaN
ts_nan.loc[ts_nan.alco_tra==2,"alco"]=1

ts_nan.loc[ts_nan.active_tra==1,"active"]=0
ts_nan.loc[ts_nan.active_tra==0,"active"]=1
ts_nan.loc[ts_nan.active_tra==2,"active"]=np.NaN


ts_nan.drop(["smoke_tra","alco_tra","active_tra"],axis=1,inplace=True)

In [25]:
ts_med.columns , X.columns

(Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'smoke', 'alco',
        'active', 'halat', 'IMT', 'Ketle', 'Devina', 'Miller', 'Nagler',
        'Mohhameda', 'Hamvi', 'Kuper', 'Ketle_cat', 'Devina_cat', 'Nagler_cat',
        'Mohhameda_cat', 'Hamvi_cat', 'Kuper_cat', 'gluc mean',
        'cholesterol mean', 'ohe1', 'ohe2', 'ohe3', 'ohe4', 'ohe5', 'ohe6'],
       dtype='object'),
 Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'smoke', 'alco',
        'active', 'cardio', 'halat', 'IMT', 'Ketle', 'Devina', 'Miller',
        'Nagler', 'Mohhameda', 'Hamvi', 'Kuper', 'Ketle_cat', 'Devina_cat',
        'Nagler_cat', 'Mohhameda_cat', 'Hamvi_cat', 'Kuper_cat', 'gluc mean',
        'cholesterol mean', 'ohe1', 'ohe2', 'ohe3', 'ohe4', 'ohe5', 'ohe6'],
       dtype='object'))

In [26]:
X["desiatok"]=(X.age//(365.25*10))
ts["desiatok"]=(ts.age//(365.25*10))
ts_med["desiatok"]=(ts.age//(365.25*10))
ts_nan["desiatok"]=(ts.age//(365.25*10))

In [27]:
X["ap_hi_desu"]=X.ap_hi//10
X["ap_lo_desu"]=X.ap_lo//10
ts_med["ap_hi_desu"]=ts_med.ap_hi//10
ts_med["ap_lo_desu"]=ts_med.ap_lo//10
ts_nan["ap_hi_desu"]=ts_nan.ap_hi//10
ts_nan["ap_lo_desu"]=ts_nan.ap_lo//10

In [28]:
set(X.columns)-set(ts_med.columns)

{'cardio'}

In [29]:
def gladmean(X,test,base,target,alf):
    #alf=25
    for i in base:
        for j in (X[i].unique()):
            mean_likehood=(X.loc[X[i]==j,target].mean()*len(X.loc[X[i]==j,target])+alf*X[target].mean())/(len(X.loc[X[i]==j,target])+alf)
            X.loc[X[i]==j,"{}_glad_mean".format(i)]=mean_likehood
            test.loc[test[i]==j,"{}_glad_mean".format(i)]=mean_likehood
    return "I_love_trains"

In [30]:
X.columns

Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'smoke', 'alco',
       'active', 'cardio', 'halat', 'IMT', 'Ketle', 'Devina', 'Miller',
       'Nagler', 'Mohhameda', 'Hamvi', 'Kuper', 'Ketle_cat', 'Devina_cat',
       'Nagler_cat', 'Mohhameda_cat', 'Hamvi_cat', 'Kuper_cat', 'gluc mean',
       'cholesterol mean', 'ohe1', 'ohe2', 'ohe3', 'ohe4', 'ohe5', 'ohe6',
       'desiatok', 'ap_hi_desu', 'ap_lo_desu'],
      dtype='object')

In [31]:
cat_feat=["desiatok",'ap_hi_desu', 'ap_lo_desu', 'Ketle_cat', 'Devina_cat',
       'Nagler_cat', 'Mohhameda_cat', 'Hamvi_cat', 'Kuper_cat']

In [32]:
%%time
gladmean(X,ts_nan,cat_feat,"cardio",25)

CPU times: user 23.2 s, sys: 144 ms, total: 23.4 s
Wall time: 23.4 s


'I_love_trains'

In [33]:
X.head()

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,...,ap_lo_desu,desiatok_glad_mean,ap_hi_desu_glad_mean,ap_lo_desu_glad_mean,Ketle_cat_glad_mean,Devina_cat_glad_mean,Nagler_cat_glad_mean,Mohhameda_cat_glad_mean,Hamvi_cat_glad_mean,Kuper_cat_glad_mean
0,18393,2,168.0,62.0,110.0,80.0,0,0,1,0,...,8.0,0.518784,0.251990,0.427408,0.406367,0.363793,0.358543,0.374008,0.355933,0.355653
1,20228,1,156.0,85.0,140.0,90.0,0,0,1,1,...,9.0,0.518784,0.814667,0.749880,0.597015,0.653310,0.654184,0.675434,0.648716,0.649954
2,18857,1,165.0,64.0,130.0,70.0,0,0,0,1,...,7.0,0.518784,0.598981,0.294007,0.406697,0.451200,0.453416,0.406612,0.453066,0.445089
3,17623,2,169.0,82.0,150.0,100.0,0,0,1,1,...,10.0,0.380164,0.857464,0.840183,0.582089,0.498257,0.494951,0.542354,0.496344,0.501673
4,17474,1,156.0,56.0,100.0,60.0,0,0,0,0,...,6.0,0.380164,0.197922,0.243515,0.377149,0.451200,0.453416,0.406612,0.453066,0.445089


In [34]:
#X["Brock"]=np.where(X.gender==1,np.log(X.weight)-np.log((X.height-110)*1.15),np.log(X.weight)-np.log((X.height-100)*1.15))

In [35]:
X.loc[X["smoke"]!=1,"cardio"].mean()

0.50210712663126067

In [36]:
print(X[X["gender"]==1]["weight"].mean())#1-женщины
X[X["gender"]==2]["weight"].mean()

72.5887546673


77.264662852472412

In [37]:
ts_piz.columns

NameError: name 'ts_piz' is not defined

In [ ]:
stack=np.vstack((np.loadtxt("./ans"),
np.loadtxt("./ansmean"),
np.loadtxt("./ansmed"),
np.loadtxt("./anspiz"),
np.loadtxt("./bayesgb"),
np.loadtxt("./dartgb"),
np.loadtxt("./deepgb"),
np.loadtxt("./gbansmean"),
np.loadtxt("./mean_xgb_and_light"),
np.loadtxt("./mean_xgb_dart_and_gb"),
np.loadtxt("./wide_gb"),
np.loadtxt("./wide_gb_nan")))

In [32]:
stack2=np.loadtxt("./ans")*np.loadtxt("./ansmean")*np.loadtxt("./ansmed")*np.loadtxt("./anspiz")*np.loadtxt("./bayesgb")*np.loadtxt("./dartgb")*np.loadtxt("./deepgb")*np.loadtxt("./gbansmean")*np.loadtxt("./mean_xgb_and_light")*np.loadtxt("./mean_xgb_dart_and_gb")*np.loadtxt("./wide_gb")*np.loadtxt("./wide_gb_nan")

In [34]:
np.power(stack2,1/12)

array([ 0.47105432,  0.56955034,  0.41333531, ...,  0.44029051,
        0.30940464,  0.73441716])

In [14]:
stack

array([[ 0.47995551,  0.50020607,  0.42691631, ...,  0.45929661,
         0.29952247,  0.7652167 ],
       [ 0.47794157,  0.50237387,  0.42599817, ...,  0.4602615 ,
         0.29691322,  0.76351134],
       [ 0.47794157,  0.57248463,  0.42599817, ...,  0.4602615 ,
         0.29691322,  0.7600548 ],
       ..., 
       [ 0.48744249,  0.57829285,  0.43122333, ...,  0.45601168,
         0.31616113,  0.7450313 ],
       [ 0.41813424,  0.66980177,  0.37080353, ...,  0.3951433 ,
         0.31172273,  0.73932701],
       [ 0.41813424,  0.66980177,  0.37080353, ...,  0.3951433 ,
         0.31172273,  0.73932701]])

In [30]:
stack.mean(axis=0,stack.shape,y.shape)

array([ 0.47171462,  0.57210182,  0.41406605, ...,  0.44128678,
        0.30959037,  0.73653485])

In [38]:
est=catboost.CatBoostClassifier()

In [39]:

model_selection.cross_val_score(est,X.drop("cardio",axis=1),y)

TypeError: expected bytes, str found

In [103]:
base_cols=["height","weight","ap_hi","ap_lo","IMT",'Ketle']

In [104]:
def transform_data(data):
    for i1, col1 in enumerate(base_cols):
        data[col1 + '_log'] = np.log(data[col1] + 1.1)

        for i2, col2 in enumerate(base_cols):
            data['%s_%s_1' % (col1, col2)] = data[col1] - data[col2]
            data['%s_%s_2' % (col1, col2)] = data[col1] + data[col2]
            data['%s_%s_3' % (col1, col2)] = data[col1] / (data[col2] + 0.1)
            data['%s_%s_4' % (col1, col2)] = data[col1] * data[col2]

            data['%s_%s_11' % (col1, col2)] = data[col1] - np.log(data[col2] + 1)
            data['%s_%s_22' % (col1, col2)] = data[col1] + np.log(data[col2] + 1)
            data['%s_%s_33' % (col1, col2)] = data[col1] / (np.log(data[col2] + 1) + 0.1)
            data['%s_%s_44' % (col1, col2)] = data[col1] * np.log(data[col2] + 1)

    return 1

In [53]:
%%time
transform_data(X)

CPU times: user 1.01 s, sys: 84 ms, total: 1.09 s
Wall time: 1.09 s


1

In [105]:
%%time
transform_data(ts_nan)

CPU times: user 584 ms, sys: 36 ms, total: 620 ms
Wall time: 620 ms


1

In [56]:
xdata=xgboost.DMatrix(X,label=y)

In [57]:
    params = {
        'eta': 0.005,
        'silent': 1,
        'eval_metric': 'logloss',
        'verbose_eval': True
    }


In [49]:
%%time
ad=xgboost.cv(params,xdata,num_boost_round=1000000,early_stopping_rounds=25,nfold=2)

CPU times: user 1h 25min 42s, sys: 8.46 s, total: 1h 25min 50s
Wall time: 24min 4s


In [50]:
ad["test-logloss-mean"][-1:]

846    0.539739
Name: test-logloss-mean, dtype: float64

In [51]:
len(ad["test-logloss-mean"])

847

In [60]:
bs

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,halat,...,Ketle_IMT_33,Ketle_IMT_44,Ketle_Ketle_1,Ketle_Ketle_2,Ketle_Ketle_3,Ketle_Ketle_4,Ketle_Ketle_11,Ketle_Ketle_22,Ketle_Ketle_33,Ketle_Ketle_44
0,18888,1,154.0,85.0,130.0,80.0,0.0,0.0,1.0,0,...,14.890931,199.065885,0.0,110.389610,0.998192,3046.466520,51.165981,59.223630,13.368165,222.370173
1,19042,2,170.0,69.0,130.0,90.0,0.0,0.0,1.0,0,...,12.247947,130.445745,0.0,81.176471,0.997542,1647.404844,36.860418,44.316053,10.603493,151.305527
2,20432,1,160.0,70.0,120.0,75.0,0.0,0.0,0.0,0,...,12.701753,146.317786,0.0,87.500000,0.997719,1914.062500,39.948909,47.551091,11.214810,166.297751
3,18133,2,185.0,94.0,130.0,90.0,0.0,0.0,1.0,0,...,14.733383,170.149446,0.0,101.621622,0.998036,2581.738495,46.863212,54.758410,12.553322,200.580697
4,16093,2,170.0,76.0,120.0,80.0,0.0,0.0,1.0,0,...,13.122552,147.833321,0.0,89.411765,0.997768,1998.615917,40.883655,48.528109,11.398086,170.876031
5,16712,1,160.0,68.0,120.0,80.0,0.0,0.0,0.0,0,...,12.439791,140.949387,0.0,85.000000,0.997653,1806.250000,38.727239,46.272761,10.974083,160.342340
6,15873,2,170.0,68.0,120.0,80.0,0.0,0.0,0.0,0,...,12.121679,127.994915,0.0,80.000000,0.997506,1600.000000,36.286428,43.713572,10.488854,148.542883
7,20424,1,164.0,60.0,120.0,80.0,0.0,0.0,0.0,0,...,11.261183,115.200113,0.0,73.170732,0.997274,1338.488995,32.958751,40.211981,9.817319,132.681028
8,20262,1,170.0,68.0,110.0,70.0,0.0,0.0,1.0,0,...,12.121679,127.994915,0.0,80.000000,0.997506,1600.000000,36.286428,43.713572,10.488854,148.542883
9,21737,1,162.0,88.0,180.0,100.0,0.0,1.0,1.0,0,...,14.915681,192.397941,0.0,108.641975,0.998162,2950.769700,50.307835,58.334140,13.206656,217.998400


In [58]:
%%time
bs=xgboost.train(params,xdata,num_boost_round=847)


CPU times: user 1h 15min 57s, sys: 2.66 s, total: 1h 16min
Wall time: 20min 4s


In [106]:
tes=xgboost.DMatrix(data=ts_nan,missing=np.NaN)

In [107]:
len(xdata.feature_names)

338

In [108]:
len(tes.feature_names)

338

In [109]:
an=bs.predict(tes)

In [61]:
an

array([ 0.41813424,  0.66980177,  0.37080353, ...,  0.3951433 ,
        0.31172273,  0.73932701], dtype=float32)

In [ ]:
%%time
bsd=xgboost.train(params1,xdata,num_boost_round=len(ad["test-logloss-mean"]))
an_d=bsd.predict(xgboost.DMatrix(ts_med))

In [ ]:
sub[0]=an_d
sub.to_csv("./dartgb",header=None,index=False)

In [ ]:
q=np.vstack((an,an_d))

In [ ]:
q.mean(axis=0)

In [110]:
sub[0]=an
sub.to_csv("./wide_gb_nan",header=None,index=False)

In [11]:
    params = {
        'eta': 0.05,
        'silent': 1,
        'eval_metric': 'logloss',
        'verbose_eval': True
    }

In [12]:
xdata=xgboost.DMatrix(stack,label=y)

In [15]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)


    cv_result = xgboost.cv(params, xdata, num_boost_round=num_rounds, nfold=2,
             callbacks=[xgboost.callback.early_stop(4)])

    return -cv_result['test-logloss-mean'].values[-1]


In [16]:
    num_rounds = 3000
    random_state = 2016
    num_iter = 175
    init_points = 25

In [17]:
from bayes_opt import BayesianOptimization

In [18]:
xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (0.1, 1),
                                            'max_depth': (5, 15),
                                            'subsample': (0.5, 1),
                                            'gamma': (0, 10),
                                            'alpha': (0, 10),
                                                })

In [19]:
xgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
---------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 4 rounds.
Stopping. Best iteration:
[0]	train-logloss:0.693147+0	test-logloss:0.693147+0

    1 | 00m00s |   -0.69315 |    9.7094 |             0.1976 |    6.9719 |     10.0649 |             3.2618 |      0.5278 | 
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 4 rounds.
Stopping. Best iteration:
[0]	train-logloss:0.693147+0	test-logloss:0.693147+0

    2 | 00m00s |   -0.69315 |    9.5160 |             0.2798 |    6.7960 |     11.6012 |             9.3715 |      0.7894 | 
Multiple eval metrics have bee

Will train until test-logloss hasn't improved in 4 rounds.
Stopping. Best iteration:
[0]	train-logloss:0.693147+0	test-logloss:0.693147+0

   23 | 00m00s |   -0.69315 |    8.7263 |             0.6106 |    5.2072 |      7.6310 |             7.4450 |      0.8936 | 
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 4 rounds.
Stopping. Best iteration:
[0]	train-logloss:0.693147+0	test-logloss:0.693147+0

   24 | 00m00s |   -0.69315 |    9.1090 |             0.3733 |    2.6401 |      7.9286 |            16.9581 |      0.8562 | 
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 4 rounds.
Stopping. Best iteration:
[0]	train-logloss:0.693147+0	test-logloss:0.693147+0

   25 | 00m00s |   -0.69315 |    4.4388 |             0.7744 |    1.9776 |      5.9419 |            12.6248 |      0.9219 | 


KeyboardInterrupt: 

In [ ]:
#122 | 02m37s |    4.5246 |             0.9205 |    1.6368 |      6.9617 |            11.0519 |      0.8381 |
#80 | 02m50s |  5.3150 |             0.5335 |    0.7957 |      5.4538 |            19.4719 |      0.9040 | 
#9.3380 |             0.1334 |    1.3801 |     10.4161 |             2.0514 |      0.8493 | 


In [23]:
params = {
        'eta': 0.000001,
        'silent': 1,
        'eval_metric': 'logloss',
        'verbose_eval': True
    }

In [24]:
%%time
ad=xgboost.cv(params,xdata,num_boost_round=1000000,early_stopping_rounds=25,nfold=2)

KeyboardInterrupt: 

In [22]:
ad

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.693147,0.0,0.693147,0.0


In [ ]:
%%time
bsd=xgboost.train(params,xdata,num_boost_round=len(ad["test-logloss-mean"]))
an_d=bsd.predict(xgboost.DMatrix(ts_med))

In [39]:
sub[0]=stack.mean(axis=0)
sub.to_csv("./globalmeancrazyidiotwhatareuduing",header=None,index=False)

In [38]:
stack.mean(axis=0)

array([ 0.47171462,  0.57210182,  0.41406605, ...,  0.44128678,
        0.30959037,  0.73653485])